In [1]:
# Makes print and division act like Python 3
from __future__ import print_function, division

# Import the usual libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Enable inline plotting at lower left
%matplotlib inline

from IPython.display import display, Latex, clear_output
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
import pynrc
from pynrc import nrc_utils
from pynrc.nrc_utils import S
from pynrc.obs_nircam import model_to_hdulist
pynrc.setup_logging('WARNING', verbose=False)

In [3]:
#                source,    dist,  fov, age,  sptype, kmag
args_sources = [('HD10647',  17.43, 14, 1000, 'F9V', 4.3),
                ('HD107146', 27.50, 14,  150, 'G2V', 5.5),
                ('HD181327', 51.80,  7,   12, 'F6V', 5.9),
                ('HD61005',  34.50,  7,  100, 'G8V', 6.5),
                ('HD32297',  113.0,  7,   30, 'A0V', 7.6)]

ref_sources  = [('iotHor',   'F8V',  4.1),
                ('HD111398', 'G5V',  5.5),
                ('HR7297',   'F7V',  5.1),
                ('HD56161',  'G5IV', 4.9),
                ('HD31411',  'A0V',  6.4)]

In [4]:
bp_k = S.ObsBandpass('k')

## ND Target Acquisition Times

In [5]:
nrc = pynrc.NIRCam(filter='F335M', pupil='CIRCLYOT', ND_acq=True,
                   wind_mode='WINDOW', xpix=128, ypix=128)

for i in range(len(args_sources)):

    name_sci, dist_sci, fov, age, spt_sci, kmag_sci = args_sources[i]
    name_ref, spt_ref, kmag_ref = ref_sources[i]

    # Science target and reference star spectra
    sp_sci = pynrc.stellar_spectrum(spt_sci, kmag_sci, 'vegamag', bp_k, catname='ck04models')
    sp_ref = pynrc.stellar_spectrum(spt_ref, kmag_ref, 'vegamag', bp_k, catname='ck04models')
    sp_sci.name = name_sci
    sp_ref.name = name_ref
    
    
    print(name_sci)

    ng_levels = [9,17,33]
    for sp in [sp_sci, sp_ref]:
        for ng in ng_levels: 
            pynrc.setup_logging('ERROR', verbose=False)
            tbl = nrc.ramp_optimize(sp, snr_goal=50, snr_frac=0.01, well_frac_max=0.5, verbose=False,
                                    ng_min=ng, ng_max=ng, nint_max=1, patterns=['RAPID'])
            nrow = len(tbl)
            if nrow>0:
                print(tbl)
                break

    print('')

pynrc.setup_logging('WARNING', verbose=False)

HD10647
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
RAPID         9    1      1.64      1.64      1.83     85.7    0.028   63.390
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
RAPID         9    1      1.64      1.64      1.83     95.3    0.033   70.472

HD107146
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
RAPID        17    1      3.09      3.09      3.28     70.1    0.018   38.714
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
RAPID        17    1      3.09      3.09      3.28     70.1    0.018   38.714

HD181327
 Pattern   NGRP NINT   t_int     t_e

## Astrometric Confirmation Settings

Want to be sensitive to objects with brightnesses of 10-18 mags at 3.3um

In [6]:
nrc = pynrc.NIRCam(filter='F335M', pupil='CIRCLYOT', wind_mode='FULL')

In [7]:
sp_dim = pynrc.stellar_spectrum('flat', 17, 'vegamag', nrc.bandpass)
sp_bright = pynrc.stellar_spectrum('flat', 11, 'vegamag', nrc.bandpass)
nrc = pynrc.NIRCam(filter='F335M', pupil='CIRCLYOT', wind_mode='FULL')
tbl = nrc.ramp_optimize(sp_dim, sp_bright, snr_goal=30, snr_frac=0.1, ng_min=2, ng_max=10, nint_min=3,
                        well_frac_max=1.2, patterns=['RAPID', 'BRIGHT1', 'SHALLOW2'], verbose=True)


BRIGHT1
RAPID
SHALLOW2
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
BRIGHT1       3    3     53.68    161.05    193.27     31.5    1.040    2.263
BRIGHT1       3    4     53.68    214.74    257.69     36.3    1.040    2.263
RAPID         4    4     42.95    171.79    214.74     29.3    0.832    2.001
RAPID         4    5     42.95    214.74    268.42     32.8    0.832    2.001
RAPID         4    6     42.95    257.68    322.11     35.9    0.832    2.001
